In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, refine_landmarks=True)

# Load filter images (make sure they have transparent backgrounds)
sunglasses = cv2.imread('googles.png', cv2.IMREAD_UNCHANGED)  # Sunglasses image with alpha channel (RGBA)
star = cv2.imread('star.png', cv2.IMREAD_UNCHANGED)  # Star image with alpha channel (RGBA)

def overlay_filter(image, filter_img, position, size):
    # Resize filter image
    filter_img = cv2.resize(filter_img, size)

    # Extract RGBA channels from the filter image
    alpha_s = filter_img[:, :, 3] / 255.0
    alpha_l = 1.0 - alpha_s

    x, y = position

    # Overlay the filter on the original image
    for c in range(0, 3):
        image[y:y + filter_img.shape[0], x:x + filter_img.shape[1], c] = (
            alpha_s * filter_img[:, :, c] + alpha_l * image[y:y + filter_img.shape[0], x:x + filter_img.shape[1], c]
        )

    return image

def apply_filters(image, landmarks):
    h, w, _ = image.shape

    # Landmark points for sunglasses (using eyes region)
    left_eye = landmarks[33]
    right_eye = landmarks[263]

    # Calculate the size and position for sunglasses
    googles_width = int(abs(right_eye.x - left_eye.x) * w * 2)  # Adjust size
    googles_height = int(googles_width * 0.4)  # Adjust proportion
    x = int(left_eye.x * w) - googles_width // 4
    y = int(left_eye.y * h) - googles_height // 2

    # Overlay sunglasses filter
    image = overlay_filter(image, sunglasses, (x, y), (googles_width, googles_height))

    # Landmark point for the star filter (above forehead)
    forehead = landmarks[10]
    star_size = 50  # Size of the star
    x = int(forehead.x * w) - star_size // 2
    y = int(forehead.y * h) - star_size - 30  # Adjust the position above forehead

    # Overlay star filter
    image = overlay_filter(image, star, (x, y), (star_size, star_size))

    return image

def capture_image_and_apply_filters():
    # Initialize webcam
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open video capture.")
        return

    print("Press 's' to capture an image.")

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to grab frame.")
            break

        cv2.imshow('Webcam', frame)

        # Capture image on 's' key press
        key = cv2.waitKey(1)
        if key == ord('s'):
            print("Image captured.")
            break

    cap.release()
    cv2.destroyAllWindows()

    # Convert the frame to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect facial landmarks
    results = face_mesh.process(rgb_frame)
    if results.multi_face_landmarks:
        landmarks = results.multi_face_landmarks[0].landmark
        frame = apply_filters(frame, landmarks)

    # Display the final image with filters
    cv2.imshow('Filtered Image', frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

if __name__ == "__main__":
    capture_image_and_apply_filters()
    


Press 's' to capture an image.
Image captured.


C:\Users\jatin\AppData\Roaming\Python\Python311\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Press 's' to capture an image.
Image captured.


C:\Users\jatin\AppData\Roaming\Python\Python311\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
